<a href="https://colab.research.google.com/github/Psousace/Diversos/blob/main/BOT_PAULO_TELEGRAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread
import time
import warnings
import requests
import xmltodict
import json
import re
import telegram
import gspread
import tweepy
import pandas as pd
import numpy as np
import datetime as dt
from statistics import median, stdev
from tqdm import tqdm
from urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)  # o ideal era não precisar disso
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
floodTlg = False

def msgTS(texto):
    agora = dt.datetime.now()
    print(f'{agora.strftime("%Y-%m-%d %H:%M:%S")} {texto}')
    return

def salvajson(name, _file=''):
    _file = _file if _file else name
    with open(f'./apoio/{_file}.json', 'w', encoding='utf-8') as f:
        json.dump(name, f, ensure_ascii=False, indent=2)
    return True

def carregaJson(_file):
    with open(f'./apoio/{_file}.json') as j:
        return json.load(j)

def sendTelegram(msg, chat_id):
    msg = msg.replace('~', '.')
    global floodTlg
    if floodTlg:
        return False
    try:
        # pass
        # print(f'Telegram {chat_id}')
        # print(msg, '\n')
        bot.sendMessage(chat_id=chat_id, text=msg, parse_mode='HTML', disable_web_page_preview=False)
    except Exception as e:
        msgTS(f'Erro Telegram\t{e}')
        if 'Flood' in str(e):
            floodTlg = True
        err = True
        return False
    return True

def clientTwitter():
    return tweepy.Client(bearer_token=twt['bearerToken'],
                         consumer_key=twt['apiKey'],
                         consumer_secret=twt['apiSecret'],
                         access_token=twt['accessToken'],
                         access_token_secret=twt['accessSecret'])

def sendTweet(msgTwitter, url=''):
    global manyTweets
    if manyTweets:
        return True
    passTime = int((time.perf_counter() - twTimer) * 1000)
    if passTime < 250:
        time.sleep((300 - passTime) / 1000)
    msgTwitter = msgTwitter.replace('Fiagro-', '#Fiagro ').replace('.', ',').replace('~', '.')
    #  Conteúdo de texto do Tweet: 280 caracteres.
    #  Observação: cada link usado ocupa 23 caracteres; se for usado um link, restarão 257 caracteres para o conteúdo de texto do Tweet.)
    tamanho = len(msgTwitter) + 23
    if tamanho > 280:
        msgTS(f'Tamnho do twitter maior que 280 ({tamanho})')
        msgTwitter = msgTwitter[:255] + '\n'
    msgTwitter += url
    try:
        tweet = client.create_tweet(text=msgTwitter)
    except:
        tweet = ''
        client = clientTwitter()
    if not tweet:
        try:
            tweet = client.create_tweet(text=msgTwitter)
        except Exception as e:
            msgTS(f'Erro no tweet {ticket}\terro: {e}') # 403 Forbidden twitter repetido
            if str(e).startswith('403'):
                tweet = 'Repetido'
                msgTS('Twitter repetido')
            elif str(e).startswith('429'): # 429 Too Many Requests
                manyTweets = True
                if not verificados['manyTweets']:
                    sendTelegram('API limitou os tweets', chat_id=base["chatIdPaulo"])
                return True
            msgTS(f'Tamanho da mensagem: {len(msgTwitter) - len(url) + 23}\n{msgTwitter}')

    return True if tweet else False

def dataBr(data, diaMes=False):
    if diaMes:
        return data[-2:] + '/' + data[5:7]
    else:
        return data[-2:] + '/' + data[5:7] + '/' + data[:4]

def escritorDic(dfs):
    pTime = time.perf_counter()
    gc = gspread.service_account(json_file)
    planilha = gc.open('FI_data')
    for key, df in dfs.items():
        # if key == 'dados':
        #     continue
        leiaMe = planilha.worksheet('Leia_Me')
        sheet = planilha.worksheet(key)
        dfSheet = df if key != 'dados' else df.iloc[:, :-2]
        sheet.update([dfSheet.columns.values.tolist()] + dfSheet.values.tolist())
        msgTS(f'Exportado para a aba {key}')
        ts = str(dt.datetime.now())[:19] # timestamp
        if key == 'dados_old':
            links = []
            for _, item in df.iterrows():
                try:
                    classeFi = dfFiB3.loc[dfFiB3['ticket'] == item[0], 'tipo'].reset_index().iloc[0,1]
                except:
                    classeFi = "FII"
                tipo = 'fiagros' if classeFi == 'FIAGRO' else 'fundos-imobiliarios'
                links.append([f'=HYPERLINK("https://statusinvest.com.br/{tipo}/{item[0].lower()}";"{item[0]}")'])
            # for ticket in df['codPregao'].tolist():
            #     links.append([f'=HYPERLINK("https://statusinvest.com.br/fundos-imobiliarios/{ticket.lower()}";"{ticket}")'])
            sheet.update(f'A2:A{len(links)+1}', links, value_input_option='USER_ENTERED')
            leiaMe.update(f'B1', ts) # timestamp
        elif key == 'dados':
            links = []
            for _, item in df.iterrows():
                try:
                    classeFi = dfFiB3.loc[dfFiB3['ticket'] == item[0], 'tipo'].reset_index().iloc[0,1]
                except:
                    classeFi = ''
                tipo = 'fiagros' if classeFi == 'FIAGRO' else 'fundos-imobiliarios'
                links.append([f'=HYPERLINK("https://statusinvest.com.br/{tipo}/{item[0].lower()}";"{item[0]}")'])
            sheet.update(f'A2:A{len(links)+1}', links, value_input_option='USER_ENTERED')
            leiaMe.update(f'B1', ts) # timestamp
            if rgToPlan: # cria links para os RG
                links = []
                for ticket, info in df['rg'].items():
                    if (info == '-'):
                        links.append([info])
                    else:
                        cod = info[:6]
                        ref = info[-7:]
                        links.append([f'=HYPERLINK("https://fnet.bmfbovespa.com.br/fnet/publico/exibirDocumento?id={cod}&cvm=true";"RG-{ref}")'])
                sheet.update(f'AE2:AE{len(links)+2}', links, value_input_option='USER_ENTERED')
                leiaMe.update(f'B4', ts) # timestamp

            links = []
            for ticket, item in df.loc[:, ['codPregao', 'cnpj']].set_index('codPregao', drop=True).iterrows():  # cria links para pesquisa
                cnpjAjustado = item[0].replace('/', '%2F')
                urlFnet = 'https://fnet.bmfbovespa.com.br/fnet/publico/abrirGerenciadorDocumentosCVM?d=2&s=0&l=10&o%5B0%5D%5BdataEntrega%5D=desc'
                urlFnet += f'idCategoriaDocumento=0&idTipoDocumento=0&idEspecieDocumento=0&situacao=A&cnpj={cnpjAjustado}'
                links.append([f'=HYPERLINK("{urlFnet}";"{ticket[:4]}_fnet")'])
            sheet.update(f'AF2:AF{len(links)+2}', links, value_input_option='USER_ENTERED')
            leiaMe.update(f'B2', ts) # timestamp
        elif key == 'renda':
            leiaMe.update(f'B3', ts) # timestamp

    pTime = str(dt.timedelta(seconds=time.perf_counter()-pTime))
    msgTS(f'Em {pTime}')

def fazUltima(df):
    df['ultima'] = 0
    cols = df.columns
    df.loc[df.iloc[:, -2] > 0, 'ultima'] = df.loc[df.iloc[:, -2] > 0, cols[-2]]
    try:
        df.loc[df['ultima'] == 0, 'ultima'] = df.loc[df['ultima'] == 0, cols[-3]]
    except:
        df.loc[df['ultima'] == 0, 'ultima'] = df.loc[df['ultima'] == 0, cols[-2]] # menos de 3 colunas

    return df

def mesRef(t=dt.date.today()):
    # t = datetime.now() #((pytz.timezone('America/Sao_Paulo')))
    if t.month == 1:
        ref = '12/' + str(t.year - 1)
    elif t.month < 11:
        ref = '0' + str(t.month - 1) + '/' + str(t.year)
    else:
        ref = str(t.month - 1) + '/' + str(t.year)
    return ref

def testeFloat(num):
    try:
        numFloat = float(num)
    except:
        numFloat = 0.
    return numFloat

def cotacao(ticket):
    url = f'https://www.google.com/finance/quote/{ticket}:BVMF'
    try:
        r = requests.get(url, headers=headers, timeout=5)
        res = re.findall(r'<div class="[^"]*YMlKec fxKbKc[^"]*?">(.*?)<\/div>', r.text)
        # print(f'\nCota Bruta - {ticket} {res}')
        cota = float(res[0][2:])
    except:
        cota = 0
    # Acha min e max
    if cota:
        a = re.findall(r'<div class="[^"]*P6K39c[^"]*?">(.*?)<\/div>', r.text)
        minMax = re.findall("\d+\.\d+", a[1])
    return cota

def getUrl(blockSize, d=1, s=0, tipoFundo=1):
    # dataInicial = '13/09/2021'
    # dataFinal = '30/09/2021'
    url = f'https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados?d={d}&s={s}&l={blockSize}&o[0][dataEntrega]=desc'
    url = f'https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados?d={d}&s={s}&l={blockSize}&o[0][dataEntrega]=asc' # contínuo
    url += f'&tipoFundo={tipoFundo}'
    # url += '&idCategoriaDocumento=7&idTipoDocumento=9&idEspecieDocumento=0'  # 14 41 Rendimentos
    url += '&idCategoriaDocumento=0&idTipoDocumento=0&idEspecieDocumento=0'
    # url += '&situacao=A'
    url += '&ultimaDataReferencia=true'
    url += f'&dataInicial={dataInicial}&dataFinal={dataFinal}'
    url += f'&_={int(time.time_ns() /1000000)}'
    return url

def executaListados(item, dfDadosFundo):
    categoriasNulas = ['Relatórios', 'Outras Informações', 'Assembleia', 'Atos de Deliberação do Administrador', 'Aviso aos Cotistas - Estruturado',
                       'Comunicado ao Mercado ', 'Informes Periódicos']
    nomeDoc = '' if item['categoriaDocumento'].strip() in categoriasNulas else item['categoriaDocumento'].strip()
    nomeDoc += item['tipoDocumento'].strip() + item['especieDocumento'].strip()
    if item['tipoDocumento'].strip() == 'Relatório Gerencial':
        pass

    return dfDadosFundo

def getInformeMensal(msg, dfDadosFundo, dfValorPatrimonialCota, dfAlavancagem):
    if item['tipoDocumento'].strip() == 'Informe Mensal Estruturado' and item['dataReferencia'] == dataRef:
        r = requests.get(urlDw, headers=headers, verify=False)
        doc = xmltodict.parse(r.content)
        competencia = doc['DadosEconomicoFinanceiros']['DadosGerais']['Competencia'][:7]
        if competencia not in dfValorPatrimonialCota.columns:
            dfValorPatrimonialCota[competencia] = 0
            dfAlavancagem[competencia] = 0
        nome = doc['DadosEconomicoFinanceiros']['DadosGerais']['NomeFundo']
        nome = '-' if not nome else nome
        try:
            segmento = doc['DadosEconomicoFinanceiros']['DadosGerais']['Autorregulacao']['SegmentoAtuacao']
        except:
            segmento = ''
        # nome = nome.encode("ascii","ignore")
        fundo = ticket
        cnpj = doc['DadosEconomicoFinanceiros']['DadosGerais']['CNPJFundo']
        cnpj = f'{cnpj[:2]}.{cnpj[2:5]}.{cnpj[5:8]}/{cnpj[8:12]}-{cnpj[-2:]}'
        publico = doc['DadosEconomicoFinanceiros']['DadosGerais']['PublicoAlvo']
        publico = publico.replace('Investidores em ', '').replace('Investidor ', '')
        gestao = doc['DadosEconomicoFinanceiros']['DadosGerais']['Autorregulacao']['TipoGestao']
        gestao = "-" if gestao == _null else gestao # _null = ('@xsi:nil', 'true')
        try:
            cotistas = int(doc['DadosEconomicoFinanceiros']['InformeMensal']['Cotistas']['@total'])
        except:
            cotistas = 0
        inicio = doc['DadosEconomicoFinanceiros']['DadosGerais']['DataFuncionamento']
        cotas = testeFloat(doc['DadosEconomicoFinanceiros']['InformeMensal']['Resumo']['NumCotasEmitidas'])
        vpc = float(doc['DadosEconomicoFinanceiros']['InformeMensal']['Resumo']['ValorPatrCotas'])
        ativo = float(doc['DadosEconomicoFinanceiros']['InformeMensal']['Resumo']['Ativo'])

        dicAtivo = doc['DadosEconomicoFinanceiros']['InformeMensal']['InformacoesAtivo']
        imoveisConst = testeFloat(dicAtivo['TotalInvestido']['DireitosBensImoveis']['ImoveisRendaConstrucao']) \
                    + testeFloat(dicAtivo['TotalInvestido']['DireitosBensImoveis']['ImoveisVendaConstrucao'])
        imoveisProntos = testeFloat(dicAtivo['TotalInvestido']['DireitosBensImoveis']['ImoveisRendaAcabados']) \
                    + testeFloat(dicAtivo['TotalInvestido']['DireitosBensImoveis']['ImoveisVendaAcabados'])
        terrenos = testeFloat(dicAtivo['TotalInvestido']['DireitosBensImoveis']['Terrenos'])
        acoesCotas = testeFloat(dicAtivo['TotalInvestido']['AcoesSociedadesAtivFII']) \
                    + testeFloat(dicAtivo['TotalInvestido']['CotasSociedadesAtivFII'])
        fof = testeFloat(dicAtivo['TotalInvestido']['FII'])
        try:
            papeis = testeFloat(dicAtivo['TotalInvestido']['CRI']) + testeFloat(dicAtivo['TotalInvestido']['LCI'])
        except:
            papeis = testeFloat(dicAtivo['TotalInvestido']['CriCra']) + testeFloat(dicAtivo['TotalInvestido']['LciLca'])
        papeis += testeFloat(dicAtivo['TotalInvestido']['LIG'])
        debentures = testeFloat(dicAtivo['TotalInvestido']['Debentures'])
        caixa = testeFloat(dicAtivo['TotalNecessidadesLiq']['@total'])

        outros = ativo - imoveisConst - imoveisProntos - terrenos - acoesCotas - fof - papeis - debentures - caixa

        dicPassivo = doc['DadosEconomicoFinanceiros']['InformeMensal']['InformacoesPassivo']
        obrImoveis = testeFloat(dicPassivo['ObrigacoesAquisicaoImov'])
        secRecebiveis = testeFloat(dicPassivo['ObrigacoesSecRecebiveis'])
        outrosValoresPagar = testeFloat(dicPassivo['OutrosValoresPagar'])
        pdd = testeFloat(dicPassivo['ProvisoesContigencias'])

        alavancagem = secRecebiveis + obrImoveis + outrosValoresPagar - caixa
        alavancagem = round(alavancagem / ativo, 2) if ativo > 0 and alavancagem > 0 else 0
        obras = round(imoveisConst / ativo, 2) if ativo > 0 else 0
        predios = round(imoveisProntos / ativo, 2) if ativo > 0 else 0
        terrenos = round(terrenos / ativo, 2) if ativo > 0 else 0
        acoesCotas = round(acoesCotas / ativo, 2) if ativo > 0 else 0
        debentures = round(debentures / ativo, 2) if ativo > 0 else 0
        outrosAtiv = round(outros / ativo, 2) if ativo > 0 else 0

        outrosFI = round(fof / ativo, 2) if ativo > 0 else 0
        caixa = round(caixa / ativo, 2) if ativo > 0 else 0
        papeis = round(papeis / ativo, 2) if ativo > 0 else 0
        cotasCotista = int(round(cotas / cotistas, 0)) if cotistas > 0 else 0
        pdd_ativo = round(pdd / ativo, 2) if ativo > 0 else 0

        dados = [nome, fundo, cnpj, classeFi, competencia, inicio, gestao, publico, cotistas, cotas, \
                caixa, predios, obras, terrenos, acoesCotas, outrosFI, papeis, debentures, outrosAtiv, pdd_ativo]
        # print(dados)
        # print(dfDadosFundo.columns)
        # print(dfDadosFundo)
        if cnpj:
            dfDadosFundo.loc[ticket, dfDadosFundo.columns[:-2]] = dados # exclui última coluna do RG
            dfValorPatrimonialCota.loc[ticket, competencia] = vpc
            dfAlavancagem.loc[ticket, competencia] = alavancagem

        tipo = dfFiB3.loc[dfFiB3['ticket'] == ticket, 'subtipo'].reset_index().iloc[0,1]
        if tipo == 'FII':
            msg = f'#{ticket} Informe Mensal #{tipo}\n'
        else:
            msg = f'#{ticket} Informe Mensal\n{tipo}\n'
        lastVPC = dfValorPatrimonialCota.loc[ticket, dfValorPatrimonialCota.columns[-2]].round(2)
        vpc12 = dfValorPatrimonialCota.loc[ticket, dfValorPatrimonialCota.columns[-6]].round(2)
        # msg += f'VPC {round(vpc, 2)}; mês anterior {lastVPC}; 6 meses atrás {vpc12}\n'
        msg += f'VPC {round(vpc, 2)}'
        if lastVPC > 0 and vpc < lastVPC * 4:
            msg += f'; mês anterior {lastVPC}'
            if vpc12 > 0 and vpc < vpc12 * 4:
                msg += f'; 6 meses atrás {vpc12}'
        msg += '\n'
        cota = cotacao(ticket)
        msg = msg + f'P/VP {round(cota/vpc, 2):.2f} - cota R$ {cota:.2f}\n' if cota else msg
        msg += f'Caixa {int(caixa*100)}%\n'
        msg = msg + f'Tijolo {int(predios*100)}%\n' if predios > 0 else msg
        msg = msg + f'Obras {int(obras*100)}%\n' if obras > 0 else msg
        msg = msg + f'Terrenos {int(terrenos*100)}%\n' if terrenos > 0 else msg
        msg = msg + f'Ações ou Cotas {int(acoesCotas*100)}%\n' if acoesCotas > 0 else msg
        msg = msg + f'Debentures {int(debentures*100)}%\n' if debentures > 0 else msg
        if tipo == 'FII':
            msg = msg + f'FIIs {int(outrosFI*100)}%\n' if outrosFI > 0 else msg
            msg = msg + f'CRIs {int(papeis*100)}%\n' if papeis > 0 else msg
        else:
            msg = msg.replace('Tijolo', 'Imóveis')
            msg = msg + f'Fiagros {int(outrosFI*100)}%\n' if outrosFI > 0 else msg
            msg = msg + f'CRAs {int(papeis*100)}%\n' if papeis > 0 else msg
        msg = msg + f'Outros Ativos {int(outrosAtiv*100)}%\n' if outrosAtiv > 0 else msg
        msg = msg + f'Provisão contingências {int(pdd_ativo*100)}%\n' if pdd_ativo > 0 else msg
        msg = msg + f'Possível Alavancagem {int(alavancagem*100)}%\n' if alavancagem > 0.015 else msg
        msg += f'Investidor {publico} - Gestão {gestao} - Cotistas {cotistas}\n'
    else:
        return False, msg, dfDadosFundo, dfValorPatrimonialCota, dfAlavancagem
    msgTS(f'{ticket} Informe Mensal {item["id"]}')
    return True, msg, dfDadosFundo, dfValorPatrimonialCota, dfAlavancagem

def getInformeTrimestral(msg):
    r = requests.get(urlDw, headers=headers, verify=False)
    doc = xmltodict.parse(r.content)
    competencia = doc['DadosEconomicoFinanceiros']['DadosGerais']['Competencia'][:2]
    try:
        if competencia == '02' or competencia == '2/' or competencia == '04' or competencia == '4/':
            distribuicao = float(doc['DadosEconomicoFinanceiros']['InformeTrimestral']['DistrResultadoAcumulado']['ResultadoLiquidoDeclarado'])
            distribuicao = distribuicao if distribuicao > 20 else distribuicao * 100
            if distribuicao > 0:
                msg += f'Distribuição do resultado acumulado no semestre: {round(distribuicao, 2)}%\n'
    except:
        print(competencia)
        print(distribuicao)
        raise
    return msg

def getRG(msg, dfDadosFundo, ticketsRG):
    if item['tipoDocumento'].strip() == 'Relatório Gerencial':
        ref = item['dataReferencia'][-4:] + '-' + item['dataReferencia'][-7:-5]
        dfDadosFundo.loc[ticket, 'rg'] = str(item['id']) + '_' + ref
        dfDadosFundo.fillna('-', inplace=True)
        ticketsRG.append(ticket)
        tipo = dfFiB3.loc[dfFiB3['ticket'] == ticket, 'subtipo'].reset_index().iloc[0,1]
        if tipo == 'FII':
            msg = f"#{ticket} Relatório Gerencial #{tipo}\n"
        else:
            msg = f"#{ticket} Relatório Gerencial\n{tipo}\n"
        msgTS(f'{ticket} RG {item["id"]}')
    else:
        return False, msg, ticketsRG
    return True, msg, dfDadosFundo, ticketsRG

def getRendimentos(msg, dfRendimento, base):
    datasPagamentos = ['' ,'']
    novoForm = False

    r = requests.get(urlDw, headers=headers, verify=False)
    doc = xmltodict.parse(r.content)
    try:
        ticketRend = doc['DadosEconomicoFinanceiros']['DadosGerais']['CodNegociacaoCota']
    except:
        novoForm = True
    rendAmort = {'rendimento': {}, 'amortizacao': {}}
    competencia = ''
    msg = ''
    dtData = dt.datetime.now() + dt.timedelta(days=7)
    if novoForm:
        final = False
        for provento in doc['DadosEconomicoFinanceiros']['InformeRendimentos']['Provento']:
            if provento == 'CodISIN':
                provento = doc['DadosEconomicoFinanceiros']['InformeRendimentos']['Provento']
                final = True
            ticketRend = provento['CodNegociacao']
            ticketRend = ticketRend if len(ticketRend) > 4 else ticket
            try:
                docRendimento = provento['Rendimento']
                rend = float(docRendimento['ValorProvento']) if novoForm else float(docRendimento['ValorProventoCota'])
                data = docRendimento['DataPagamento']
                dtData = dt.datetime.strptime(data, '%Y-%m-%d')
                data = data[-2:] + '/' + data[5:7] + '/' + data[:4]
                competencia = docRendimento['DataPagamento'][:7]
                rendAmort['rendimento'].update({ticketRend: [rend, data]})
            except:
                rendAmort['rendimento'].update({ticketRend: [0, '']})
            try:
                docAmortizacao = provento['Amortizacao']
                amort = float(docAmortizacao['ValorProvento']) if novoForm else float(docAmortizacao['ValorProventoCota'])
                data = docAmortizacao['DataPagamento']
                data = data[-2:] + '/' + data[5:7] + '/' + data[:4]
                competencia = docAmortizacao['DataPagamento'][:7] if not competencia else competencia
                rendAmort['amortizacao'].update({ticketRend: [amort, data]})
            except:
                rendAmort['amortizacao'].update({ticketRend: [0, '']})
            try:
                mes = int(competencia[-2:])
            except:
                print(competencia, competencia[-2:])
                print(docRendimento)
                raise
            if mes > 10:
                competencia = competencia[:5] + str(mes - 1)
            elif mes > 1:
                competencia = competencia[:5] + '0' + str(mes - 1)
            else:
                competencia = str(int(competencia[:4]) - 1) + '-12'

            if competencia not in dfRendimento.columns:
                today = dt.datetime.now()
                proxMes = (today.replace(day=28) + dt.timedelta(days=35)).replace(day=1) - dt.timedelta(days=1) # último dia do próximo mês
                if today.day > 21 and proxMes.date() >= dtData.date() and today < dtData:
                    msgComp = f'### COMPETÊNCIA NOVA {competencia} ###'
                    print(msgComp)
                    sendTelegram(msgComp, chat_id=base["chatIdPaulo"])
                    dfRendimento[competencia] = np.nan
                else:
                    msgComp = f'### COMPETÊNCIA ANÔMALA {competencia} {ticketRend}###'
                    msgTS(msgComp)
                    print(dfRendimento.columns, dtData, proxMes.date(), dtData.date(), today)
                    sendTelegram(msgComp, chat_id=base["chatIdPaulo"])
                    return False, msg, dfRendimento, rendAmort, ticketRend, novoForm

            rendimento = rendAmort['rendimento'][ticketRend][0]
            amortizacao = rendAmort['amortizacao'][ticketRend][0]

            tipo = dfFiB3.loc[dfFiB3['ticket'] == ticket, 'subtipo'].reset_index().iloc[0,1]
            if rendimento > 0:
                dataPagamento = rendAmort["rendimento"][ticketRend][1]
                dfRendimento.loc[ticketRend, competencia] = rendimento
                if ticketRend.endswith('11') or ticketRend.endswith('11B'): #pagamento de cota e não de recibo
                    rendDec = str(f'{rendimento:.2f}') if (rendimento * 10000) % 100 == 0 else rendimento
                    if tipo == 'FII':
                        msg += f'#{ticketRend} Rendimento R$ {rendDec}\nPagamento em {dataPagamento[:5]} #{tipo}\n'
                    else:
                        msg += f'#{ticketRend} Rendimento R$ {rendDec}\nPagamento em {dataPagamento[:5]}\n{tipo}\n'

                    if dfRendimento.loc[ticketRend, dfRendimento.columns[-2]] > 0: # Verifica se mês anterior foi maior do que 0
                        rendAnterior = dfRendimento.loc[ticketRend, dfRendimento.columns[-2]]
                        rendAnterior = str(f'{rendAnterior:.2f}') if (rendAnterior * 10000) % 100 == 0 else rendAnterior
                        msg += f'Mês passado R$ {rendAnterior}\n'
                    if dfRendimento.loc[ticketRend, dfRendimento.columns[-4:]].all(skipna=True):  # Verifica se não tem 0 nos valores
                        mediaRendimento = round(dfRendimento.loc[ticketRend, dfRendimento.columns[-4:]].mean(skipna=True), 2) # calcula média
                        msg += f'Média de 4 meses R$ {mediaRendimento:.2f}\n'

                    cota = cotacao(ticketRend)
                    msg = msg + f'Yeld do mês {round(100*rendimento/cota, 2)}%\nYeld anual médio {round((100*mediaRendimento * 12)/cota, 1)}%\nCota R$ {cota:.2f}\n' if cota else msg
                else:
                    # msg = msg if msg else f'#{ticket}\n'
                    msg = msg if ticket in msg else msg + f'#{ticket}\n'
                    rendDec = str(f'{rendimento:.2f}') if (rendimento * 10000) % 100 == 0 else rendimento
                    if dataPagamento[:5] in msg:
                        msg += f'{ticketRend} R$ {rendDec}\n'
                    else:
                        msg += f'{ticketRend} Rendimento R$ {rendimento}\nPagamento em {dataPagamento[:5]} #{tipo}\n'
            elif rendimento == 0 and amortizacao == 0:
                msg += f'{ticketRend} Rendimentos e amortizações zerados\nPossível erro\n'

            if amortizacao > 0:
                dataPagamento = rendAmort["amortizacao"][ticketRend][1]
                amortDec = str(f'{amortizacao:.2f}') if (amortizacao * 10000) % 100 == 0 else amortizacao
                if msg:
                    msg += f'Amortização R$ {amortDec}\n'
                    msg = msg if dataPagamento[:5] in msg else msg + f'Pagamento em {dataPagamento[:5]}\n'
                else:
                    msg = f'#{ticketRend} Amortização R$ {amortDec} #{tipo}\n'
                    msg += f'Pagamento em {dataPagamento[:5]}\n'
            if final:
                break
    else:
        ticketRend = ticketRend if len(ticketRend) > 4 else ticket
        base = doc['DadosEconomicoFinanceiros']['InformeRendimentos']
        try:
            docRendimento = base['Rendimento']
            rend = float(docRendimento['ValorProvento']) if novoForm else float(docRendimento['ValorProventoCota'])
            data = docRendimento['DataPagamento']
            dtData = dt.datetime.strptime(data, '%Y-%m-%d')
            data = data[-2:] + '/' + data[5:7] + '/' + data[:4]
            competencia = docRendimento['DataPagamento'][:7]
            rendAmort['rendimento'].update({ticketRend: [rend, data]})
        except:
            rendAmort['rendimento'].update({ticketRend: [0, '']})
        try:
            docAmortizacao = base['Amortizacao']
            amort = float(docAmortizacao['ValorProvento']) if novoForm else float(docAmortizacao['ValorProventoCota'])
            data = docAmortizacao['DataPagamento']
            data = data[-2:] + '/' + data[5:7] + '/' + data[:4]
            competencia = docAmortizacao['DataPagamento'][:7] if not competencia else competencia
            rendAmort['amortizacao'].update({ticketRend: [amort, data]})
        except:
            rendAmort['amortizacao'].update({ticketRend: [0, '']})
        try:
            mes = int(competencia[-2:])
        except:
            print(competencia, competencia[-2:])
            print(docRendimento)
            raise
        if mes > 10:
            competencia = competencia[:5] + str(mes - 1)
        elif mes > 1:
            competencia = competencia[:5] + '0' + str(mes - 1)
        else:
            competencia = str(int(competencia[:4]) - 1) + '-12'

        if competencia not in dfRendimento.columns:
            today = dt.datetime.now()
            proxMes = (today.replace(day=28) + dt.timedelta(days=35)).replace(day=1) - dt.timedelta(days=1) # último dia do próximo mês
            if today.day > 23 and proxMes.date() >= dtData.date() and today < dtData:
                msgComp = f'### COMPETÊNCIA NOVA {competencia} ###'
                print(msgComp)
                sendTelegram(msgComp, chat_id=base["chatIdPaulo"])
                dfRendimento[competencia] = np.nan
            else:
                msgComp = f'### COMPETÊNCIA ANÔMALA {competencia} ###'
                msgTS(msgComp)
                print(dfRendimento.columns, dtData, proxMes.date(), dtData.date(), today)
                sendTelegram(msgComp, chat_id=base["chatIdPaulo"])
                return False, msg, dfRendimento, rendAmort, ticketRend, novoForm

        rendimento = rendAmort['rendimento'][ticketRend][0]
        amortizacao = rendAmort['amortizacao'][ticketRend][0]
        dataPagamento = rendAmort["rendimento"][ticketRend][1]

        tipo = dfFiB3.loc[dfFiB3['ticket'] == ticket, 'subtipo'].reset_index().iloc[0,1]
        if rendimento > 0:
            dfRendimento.loc[ticketRend, competencia] = rendimento
            if ticketRend.endswith('11') or ticketRend.endswith('11B'): #pagamento de cota e não de recibo
                if tipo == 'FII':
                    msg = f'#{ticketRend} Rendimento R$ {rendimento:.2f}\nPagamento em {dataPagamento[:5]} #{tipo}\n'
                else:
                    msg = f'#{ticketRend} Rendimento R$ {rendimento:.2f}\nPagamento em {dataPagamento[:5]}\n{tipo}\n'
                msg += f'Mês passado R$ {dfRendimento.loc[ticketRend, dfRendimento.columns[-2]]:.2f}\n'
                mediaRendimento = round(dfRendimento.loc[ticketRend, dfRendimento.columns[-4:]].mean(skipna=True), 2) # calcula média
                msg += f'Média de 4 meses R$ {mediaRendimento:.2f}\n'
                cota = cotacao(ticketRend)
                msg = msg + f'Yeld do mês {round(100*rendimento/cota, 2)}%\nYeld anual médio {round((100*mediaRendimento * 12)/cota, 1)}%\nCota R$ {cota:.2f}\n' if cota else msg
            else:
                msg = msg if ticket in msg else msg + f'#{ticket}\n'
                msg += f'{ticketRend} Rendimento R$ {rendimento:.2f}\n'
                msg = msg if dataPagamento[:5] in msg else msg + f'Pagamento em {dataPagamento[:5]} #{tipo}\n'
        elif rendimento == 0 and amortizacao == 0:
            if ticketRend.endswith('11') or ticketRend.endswith('11B'):
                msg = f'#{ticketRend} Rendimentos e amortizações zerados\nPossível erro\n'
            else:
                msg = f'#{ticketRend} Rendimentos e amortizações de {ticket} zerados\nPossível erro\n'

        if amortizacao > 0:
            if msg:
                msg += f'Amortização R$ {amortizacao:.2f}\nPagamento em {rendAmort["amortizacao"][1][:5]}\n'
            else:
                msg += f'#{ticketRend} Amortização R$ {amortizacao:.2f} #{tipo}\nPagamento em {rendAmort["amortizacao"][1][:5]}\n'
    # print(rendAmort)
    msgTS(f'{ticketRend} R$ {rendimento} {item["id"]}')
    return True, msg, dfRendimento, rendAmort, ticketRend, novoForm

def getRendimentosOld(msg, dfRendimento):
    datasPagamentos = ['' ,'']
    novoForm = False
    if item['tipoDocumento'].strip() == 'Rendimentos e Amortizações':
        r = requests.get(urlDw, headers=headers, verify=False)
        doc = xmltodict.parse(r.content)
        try:
            ticketRend = doc['DadosEconomicoFinanceiros']['DadosGerais']['CodNegociacaoCota']
        except:
            ticketRend = doc['DadosEconomicoFinanceiros']['InformeRendimentos']['Provento']['CodNegociacao']
            novoForm = True
        ticketRend = ticketRend if len(ticketRend) > 4 else ticket
        rendAmort = {'rendimento': [], 'amortizacao': []}
        competencia = ''
        if novoForm:
            base = doc['DadosEconomicoFinanceiros']['InformeRendimentos']['Provento']
        else:
            base = doc['DadosEconomicoFinanceiros']['InformeRendimentos']
        try:
            docRendimento = base['Rendimento']
            rend = float(docRendimento['ValorProvento']) if novoForm else float(docRendimento['ValorProventoCota'])
            data = docRendimento['DataPagamento']
            data = data[-2:] + '/' + data[5:7] + '/' + data[:4]
            competencia = docRendimento['DataPagamento'][:7]
            rendAmort['rendimento'] = [rend, data]
        except:
            rendAmort['rendimento'] = [0, '']
        try:
            docAmortizacao = base['Amortizacao']
            amort = float(docAmortizacao['ValorProvento']) if novoForm else float(docAmortizacao['ValorProventoCota'])
            data = docAmortizacao['DataPagamento']
            data = data[-2:] + '/' + data[5:7] + '/' + data[:4]
            competencia = docAmortizacao['DataPagamento'][:7] if not competencia else competencia
            rendAmort['amortizacao'] = [amort, data]
        except:
            rendAmort['amortizacao'] = [0, '']
        try:
            mes = int(competencia[-2:])
        except:
            print(competencia, competencia[-2:])
            print(docRendimento)
            raise
        if mes > 10:
            competencia = competencia[:5] + str(mes - 1)
        elif mes > 1:
            competencia = competencia[:5] + '0' + str(mes - 1)
        else:
            competencia = str(int(competencia[:4]) - 1) + '-12'

        if competencia not in dfRendimento.columns:
            print('### COMPETÊNCIA NOVA ###')
            dfRendimento[competencia] = np.nan

        rendimento = rendAmort['rendimento'][0]
        amortizacao = rendAmort['amortizacao'][0]
        dataPagamento = rendAmort["rendimento"][1]

        tipo = dfFiB3.loc[dfFiB3['ticket'] == ticket, 'subtipo'].reset_index().iloc[0,1]
        msg = ''
        if rendimento > 0:
            dfRendimento.loc[ticketRend, competencia] = rendimento
            if ticketRend.endswith('11') or ticketRend.endswith('11B'): #pagamento de cota e não de recibo
                if tipo == 'FII':
                    msg = f'#{ticketRend} Rendimento R$ {rendimento:.2f}\nPagamento em {dataPagamento[:5]} #{tipo}\n'
                else:
                    msg = f'#{ticketRend} Rendimento R$ {rendimento:.2f}\nPagamento em {dataPagamento[:5]}\n{tipo}\n'
                msg += f'Mês passado R$ {dfRendimento.loc[ticketRend, dfRendimento.columns[-2]]:.2f}\n'
                mediaRendimento = round(dfRendimento.loc[ticketRend, dfRendimento.columns[-4:]].mean(skipna=True), 2) # calcula média
                msg += f'Média de 4 meses R$ {mediaRendimento:.2f}\n'
                cota = cotacao(ticketRend)
                msg = msg + f'Yeld do mês {round(100*rendimento/cota, 2)}%\nYeld anual médio {round((100*mediaRendimento * 12)/cota, 1)}%\nCota R$ {cota}\n' if cota else msg
            else:
                msg = f'#{ticket}\n'
                msg += f'{ticketRend} Rendimento R$ {rendimento:.2f}\nPagamento em {dataPagamento[:5]} #{tipo}\n'
        elif rendimento == 0 and amortizacao == 0:
            if ticketRend.endswith('11') or ticketRend.endswith('11B'):
                msg = f'#{ticketRend} Rendimentos e amortizações zerados\nPossível erro\n'
            else:
                msg = f'#{ticketRend} Rendimentos e amortizações de {ticket} zerados\nPossível erro\n'

        if amortizacao > 0:
            if msg:
                msg += f'Amortização R$ {amortizacao:.2f}\nPagamento em {rendAmort["amortizacao"][1][:5]}\n'
            else:
                msg += f'#{ticketRend} Amortização R$ {amortizacao:.2f} #{tipo}\nPagamento em {rendAmort["amortizacao"][1][:5]}\n'
    else:
        return False, msg, dfRendimento, {}
    # print(rendAmort)
    msgTS(f'{ticketRend} R$ {rendimento:.2f}')
    return True, msg, dfRendimento, rendAmort, ticketRend

def getDemaiInfos():
    categoriaDoc = item["categoriaDocumento"].strip()
    tipoDoc = item["tipoDocumento"].replace('Estruturado', '').replace('()', '').strip()
    especieDoc = item["especieDocumento"].strip()
    categoriasNulas = ['Assembleia', 'Atos de Deliberação do Administrador', 'Comunicado ao Mercado', 'Informes Periódicos', 'Outras Informações',
                       'Políticas de Governança Corporativa ', 'Relatórios', 'Regulamento de Emissores B3']
    especiesNulas = ['Processo de enforcement']
    if categoriaDoc in categoriasNulas:
        categoriaDoc = ''
    elif categoriaDoc.startswith('Oferta Pública'):
        categoriaDoc = categoriaDoc[18:] # sai apenas distribuição de cotas
        categoriaDoc = 'Emissão -'
    if especieDoc in especiesNulas:
        especieDoc = ''
    nomeDoc = f'{categoriaDoc} {tipoDoc} {especieDoc}'.strip()
    if tipoFi  == 'FII':
        msg = f'#{ticket} {nomeDoc} #{tipoFi}\n'
    else:
        msg = f'#{ticket} {nomeDoc}\n{tipoFi}\n'
    msgTS(f'{ticket} {nomeDoc} {item["id"]}')
    return True, msg

def getInicioSubscricao():
    _, msg = getDemaiInfos()
    try:
        msg = msg.replace('Emissão - Formulário de Subscrição de Cotas', 'Formulário de Subscrição')
    except:
        pass
    r = requests.get(urlDw, headers=headers, verify=False)
    doc = xmltodict.parse(r.content)
    info = doc['DireitoPreferenciaSubscricaoCotas']
    precoSubs = round(float(info['DadosGerais']['PrecoSubscricao']), 2)
    # tipoOferta = info['DadosGerais']['TipoOferta']
    # msg += f'Emissão {tipoOferta[-3:]}\n'
    msg += f'Preço + taxas: R$ {precoSubs:.2f}\n'
    percentSubs = str(round(float(info['DadosGerais']['PercentualSubscricao']), 4)) + '%'
    msg += f'Proporção: {percentSubs}\n'
    dataCorte = dataBr(info['DadosGerais']['DataCorte'], diaMes=True)
    msg += f'Data de corte: {dataCorte}\n'
    inicioPref = dataBr(info['DireitoPreferencia']['ExercicioDireitoPreferenciaB3']['DtInicioPrazo'], diaMes=True)
    fimDireitoPref = dataBr(info['DireitoPreferencia']['ExercicioDireitoPreferenciaB3']['DtFimPrazo'], diaMes=True)
    msg += f'Preferência de {inicioPref} a {fimDireitoPref}\n'
    liqPref = dataBr(info['DireitoPreferencia']['DtLiquidacao'], diaMes=True)
    msg += f'Liquidação em {liqPref}\n'
    sobras = True if info['PossuiSobrasSubscricao'] == 'true' else False
    montanteAdicional = True if info['PossuiMontanteAdicional'] == 'true' else False
    op = True if info['PossuiMontanteAdicional'] == 'true' else False
    try:
        inicioSobras = dataBr(info['SobrasSubscricao']['ExercicioSobrasSubscricaoB3']['DtInicioPrazo'], diaMes=True)
        fimSobras = dataBr(info['SobrasSubscricao']['ExercicioSobrasSubscricaoB3']['DtFimPrazo'], diaMes=True)
        liqSobras = dataBr(info['SobrasSubscricao']['DtLiquidacao'], diaMes=True)
    except:
        sobras = False
    if sobras and montanteAdicional:
        msg += f'Sobras e adicional de {inicioSobras} a {fimSobras}\n'
        msg += f'Liquidação em {liqSobras}\n'
    elif sobras:
        msg += f'Sobras de {inicioSobras} a {fimSobras}\n'
        msg += f'Liquidação em {liqSobras}\n'

    # msg = msg.replace('/2022', '')
    return True, msg

def getConversaoSubscricao():
    _, msg = getDemaiInfos()
    r = requests.get(urlDw, headers=headers, verify=False)
    doc = xmltodict.parse(r.content)
    info = doc['LiberacaoNegociacao']['ListaIsin']

    try:
        dataConversao = dataBr(info['Isin'][0]['DataConversao'], diaMes=True)
        msg += f'Conversão das cotas em {dataConversao}\n'
    except:
        pass
    try:
        dataLiberacao = dataBr(info['Isin'][0]['DataLiberacaoNegociacao'], diaMes=True)
        msg += f'Liberação das cotas em {dataLiberacao}\n'
    except:
        pass
    try:
        info['Isin'].reverse()
        for item in info['Isin']:
            if item['DataLiberacaoLockUp']:
                dataLockUp = dataBr(item['DataLiberacaoLockUp'], diaMes=True)
                if int(dataLockUp[:1]) > 0:
                    msg += f'Há cotas em lock up até {dataLockUp}\n'
                    break
    except:
        pass

    captado = int(float(doc['LiberacaoNegociacao']['OutrasInformacoes']['MontanteFinal']))
    montanteCaptado = ('{:,}'.format(captado)).replace(',', '~')
    msg += f'Montante captado R$ {montanteCaptado}\n'
    return True, msg

def getProventoCarteiraOld(cotas, ticketRend):
    _data = rendAmort['rendimento'][1] if rendAmort['rendimento'][0] else rendAmort['amortizacao'][1]
    mes = _data[3:5]
    ano = _data[6:10]
    anoMes = int(ano + mes)
    investidores = ['bia', 'paulo']
    # Tem que pensar em um jeito de conferir o mês
    msg = ''
    if cotas['mes'] < anoMes:
        cotas['mes'] = anoMes
        for investidor in investidores:
            cotas[investidor]['pagos'] = []
            cotas[investidor]['amortizados'] = []
            cotas[investidor]['totalPago'] = 0
    for investidor in investidores:
        for tipo, pag in rendAmort.items():
            if not pag[0]: # valor de rendimento ou amortização
                continue
            tipoProvento = 'pagos' if tipo == 'rendimento' else 'amortizados'
            if ticketRend in cotas[investidor].keys() and ticketRend not in cotas[investidor][tipoProvento]:
                proventoAtivo = int(cotas[investidor][ticketRend] * pag[0])
                cotas[investidor]['totalPago'] += proventoAtivo
                cotas[investidor][tipoProvento].append(ticketRend)
                nome = investidor.capitalize()
                _tipo = '' if tipo == 'rendimento' else f' ({tipo})'
                msg += f"#{nome} receberá R$ {proventoAtivo} {_tipo}\nTotal no mês {nome} - R$ {cotas[investidor]['totalPago']}\n"

    return msg, cotas

def getProventoCarteira(cotas):
    tipos = ['rendimento', 'amortizacao']
    tickets = rendAmort['rendimento'].keys()
    msg = ''
    for ticket in tickets:
        for tipo in tipos:
            if not rendAmort[tipo][ticket][0]:
                continue
            _data = rendAmort[tipo][ticket][1]
            mes = _data[3:5]
            ano = _data[6:10]
            anoMes = int(ano + mes)
            investidores = ['bia', 'paulo']
            # Tem que pensar em um jeito de conferir o mês
            if cotas['mes'] < anoMes:
                cotas['mes'] = anoMes
                for investidor in investidores:
                    cotas[investidor]['pagos'] = []
                    cotas[investidor]['amortizados'] = []
                    cotas[investidor]['totalPago'] = 0
            for investidor in investidores:
                tipoProvento = 'pagos' if tipo == 'rendimento' else 'amortizados'
                if ticket in cotas[investidor].keys() and ticket not in cotas[investidor][tipoProvento]:
                    proventoAtivo = int(cotas[investidor][ticket] * rendAmort[tipo][ticket][0])
                    cotas[investidor]['totalPago'] += proventoAtivo
                    cotas[investidor][tipoProvento].append(ticket)
                    nome = investidor.capitalize()
                    _tipo = '' if tipo == 'rendimento' else f' ({tipo})'
                    msg += f"#{nome} receberá R$ {proventoAtivo} {_tipo}\nTotal no mês {nome} - R$ {cotas[investidor]['totalPago']}\n"
    return msg, cotas

def sendPessoalOld(msg, ticketRend):
    cotas = cotasAcompanhamento
    if rendimentos:
        rendimento = rendAmort['rendimento'][0]
        amortizacao = rendAmort['amortizacao'][0]
        if rendimento or amortizacao:
            msgParcial, cotas = getProventoCarteira(cotas, ticketRend)
            msg += msgParcial
    msg += f'{url}'
    resTelegram = sendTelegram(msg, chat_id=base['chatIdRelFiis'])
    return resTelegram, cotas

def sendPessoal(msg, ticketRend):
    cotas = cotasAcompanhamento
    if rendimentos and novoForm:
        msgParcial, cotas = getProventoCarteira(cotas)
        msg += msgParcial
    elif rendimentos and not novoForm:
        for ticketAtual in rendAmort['rendimento'].keys():
            rendimento = rendAmort['rendimento'][ticketAtual][0]
            amortizacao = rendAmort['amortizacao'][ticketAtual][0]
            if rendimento or amortizacao:
                msgParcial, cotas = getProventoCarteiraOld(cotas, ticketRend)
                msg += msgParcial
    msg += f'{url}'
    resTelegram = sendTelegram(msg, chat_id=base['chatIdRelFiis'])
    return resTelegram, cotas

def sendFiagro(msg, ajuste=True):
    if ajuste:
        msg = msg.replace('.', ',')
        # msg += f'<a href="{url}">Siga o link</a>'
        msg += f'{url}'
    resTelegram = sendTelegram(msg, chat_id=base['chatIdCanalFiagro'])
    return resTelegram

def sendFii(msg, ajuste=True):
    if ajuste:
        msg = msg.replace('.', ',')
        # msg += f'<a href="{url}">Siga o link</a>'
        try:
            msg += f'{url}'
        except:
            pass
    resTelegram = sendTelegram(msg, chat_id=base['chatIdCanalInfoFii'])
    return resTelegram

def verificaInfra(verificados):
    infraTimer = time.perf_counter()
    msgTS(f'Verificando FI-Infra e FIP-IE')
    dfFundos = pd.read_hdf('./apoio/dfInfraB3.h5', key='df')
    infraLinks = carregaJson('infraLinks')
    ativos = dfFundos['ticket'].to_list()
    reInfo = r"\)\W+(.*(?=- /d[2+]))"
    reInfoSemData = r"\)\W+(.*)"
    texto = ''
    dataHoje = dt.date.today().isoformat()
    url = 'https://sistemasweb.b3.com.br/PlantaoNoticias/Noticias/ListarTitulosNoticias?agencia=18'
    url += f'&palavra={texto}&dataInicial={dataHoje}&dataFinal={dataHoje}'
    r = requests.get(url, headers=headers, timeout=10)
    j = r.json()
    j.reverse()
    for item in j:
        noticiaValida = False
        res = item['NwsMsg']
        msg = res['headline']
        quebra = msg.find(")")
        ticket = msg[msg.find("(")+1:quebra] + '11'
        id1 = res['id']
        if (ticket in ativos or ticket + 'B' in ativos) and id1 not in verificados["idTwitter"]:
            msgTwitter = ''
            if not ticket in ativos:
                ticket += 'B'
            try:
                info = (re.findall(reInfo, msg))[0]
            except:
                info = (re.findall(reInfoSemData, msg))
                info = info[0] if info else msg

            tipo = dfFundos.loc[dfFundos['ticket'] == ticket, 'tipo'].values[0]
            msgTwitter = f'#{ticket} {info}\n#{tipo}\n'
            msgTS(f'{ticket} {info} {id1}')
            dataNoticia = res['dateTime'][:10]
            urlNoticia = f'https://sistemasweb.b3.com.br/PlantaoNoticias/Noticias/Detail?idNoticia={id1}&agencia=18&dataNoticia={dataNoticia}'
            urlNoticia += f'\nComunicados e outros Documentos:\n{infraLinks[ticket[:4]]}'
            noticiaValida = True
            if sendTweet(msgTwitter, url=urlNoticia):
                verificados["idTwitter"].append(id1)
                salvajson(verificados, 'verificadosB3')
            else:
                erroTicket += 1
                msgTS(f'Erro no Twitter {ticket}')
            if sendTelegram(msgTwitter + urlNoticia, chat_id=base["chatIdInfoInfra"]):
                verificados["fullCheck"].append(id1)
                salvajson(verificados, 'verificadosB3')
        if noticiaValida and ticket in fiAcompanhamento and id1 not in verificados["idPessoal"]:
            resTelegram = sendTelegram(msgTwitter + urlNoticia, chat_id=base['chatIdRelFiis'])
            if resTelegram:
                verificados["idPessoal"].append(id1)
                salvajson(verificados, 'verificadosB3')
            else:
                msgTS('Erro no Telegram - Pessoal')
                erroTicket += 1
    infraTimer = str(dt.timedelta(seconds=time.perf_counter()-infraTimer))
    msgTS(f'{len(j)} notícias da B3 analisadas em {infraTimer}')
    return verificados

tTime = time.perf_counter()
tempoInfra = dt.datetime.now()
print(' ')
urlPesquisa = f'https://fnet.bmfbovespa.com.br/fnet/publico/pesquisarGerenciadorDocumentosDados?d=1&s=0&l=10&o%5B0%5D%5BdataEntrega%5D=desc&situacao=A&cnpj=01.201.140%2F0001-90'

base = carregaJson('base')
headers = base['headers']
scopes = base['scopes']
json_file = "./apoio/credentials.json"
_null = {'@xsi:nil': 'true'}
path = './apoio/dataFIIs.h5'
dataRef = mesRef() # referência dos Informes Mensais (só processa as desse mês) mudar na virada do mês

twt = carregaJson('twitter')
twTimer = time.perf_counter()
verificados = carregaJson('verificadosB3')

hoje = dt.date.today()
now = dt.datetime.now()
diaAnalise = dt.datetime.strptime(verificados['diaAnalise'], '%d/%m/%Y').date()
repescagem = False
if hoje > diaAnalise:  # or now.minute == 0:
    verificados["lenFnet"]["fii"] = 0
    verificados["lenFnet"]["fidc"] = 0
    manyTweets = False
    repescagem = True
    msgTS('Repescagem')
# elif now.minute == 4 and (verificados["lenFnet"]["fii"] > 199 or verificados["lenFnet"]["fidc"] > 199):
#     msgTS('Consulta do 4º minuto abortada')
#     exit()
verificados["lenFnet"]["fii"] = 0 if verificados["lenFnet"]["fii"] < 20 else verificados["lenFnet"]["fii"] - 20
verificados["lenFnet"]["fidc"] = 0 if verificados["lenFnet"]["fidc"] < 20 else verificados["lenFnet"]["fidc"] - 20

manyTweets = verificados['manyTweets']
blockTlgTweet = False if verificados['manyTweets'] else True
manyTweets = False

dfFiB3 = pd.read_hdf('./apoio/dfFiB3.h5', key='df')
dfDadosFundo = pd.read_hdf('./apoio/dfDadosFundos.h5', key='df')
dfValorPatrimonialCota = pd.read_hdf('./apoio/dfVPC.h5', key='vpc')
dfAlavancagem = pd.read_hdf('./apoio/dfAlavancagem.h5', key='alavancagem')
dfRendimento = pd.read_hdf('./apoio/dfRendimento.h5', key='renda')
fiisListados = carregaJson('fiisListados')
fiisListadosOld = fiisListados.copy()
fullCheck = verificados['fullCheck']

bot = telegram.Bot(token=base['tokenPapubaTlg'])

cotasAcompanhamento = carregaJson('cotasAcompanhamento')
cotasBiaPaulo = list(set([*list(cotasAcompanhamento['bia'].keys()), *list(cotasAcompanhamento['paulo'].keys())]))
dfInfra = pd.read_hdf('./apoio/dfInfraB3.h5', key='df')

if verificados['ultimoDU'] == str(dt.date.today()): # aumenta o número de fundos no TW no último DU e não envia rend para todos
    limiteCotistas = 2500
else:
    limiteCotistas = 5000
fundosTw = [*dfDadosFundo.loc[dfDadosFundo['cotistas'] > limiteCotistas, :].index.to_list(), *cotasBiaPaulo, *dfInfra['ticket'].to_list()]

###########################################
dataInicial = verificados['diaAnalise']
dataFinal = dataInicial
# verificado[0] = 0
# verificado = [0]
############################################
msgTS(f'Data Inicial {dataInicial}\tverificado FII {verificados["lenFnet"]["fii"]} - FIDC {verificados["lenFnet"]["fidc"]}')

fiagros = dfFiB3.loc[dfFiB3['tipo'] == 'FIAGRO', 'ticket'].tolist()
lstFIIs = dfFiB3.loc[dfFiB3['tipo'] == 'FII', 'ticket'].tolist()
nomesPregao = dfFiB3.loc[:, 'nomePregao'].tolist()
tickets = dfFiB3.loc[:, ['nomePregao', 'ticket']].set_index('nomePregao').to_dict()['ticket']
fiListados = dfFiB3.loc[:, 'ticket'].tolist()
rgToPlan = False
informesToPlan = False
rendimentosToPlan = False
client = clientTwitter()

_tipoFI = {}
msg = ''
for _, row in dfFiB3.iterrows():
    ticket = row[2]
    fiisListados.update({row[1].strip(): ticket})
    nomesPregao.append(row[1].strip())
    _tipoFI.update({ticket: row[-1].strip()})
    if ticket not in list(fiisListadosOld.values()):
        msg = f'Novo #{row[-1]} listado: #{ticket}\n{row[0]}'
        msgTS(msg)
        sendTweet(msg)
        twTimer = time.perf_counter()
        if 'FIAGRO' in msg.upper():
            sendFiagro(msg.replace('#Fiagro-', 'Fiagro '), ajuste=False)
        else:
            sendFii(msg)
        salvajson(fiisListados, 'fiisListados')

msgTS(f'FI Listados {len(fiListados)}')
for tipoFundo in list(verificados['lenFnet'].keys()):

    pTime = time.perf_counter()
    dfs = {}
    codTipoFundo = 1 if tipoFundo == 'fii' else 2 # 1 FII, 2 FIDC, 10 FIP
    msgTS(f'Analisando {tipoFundo}')
    ticketsRG = []
    # if codTipoFundo == 2:
    #     "FIDC não verificado"
    #     break

    #################### Baixa dados ######################
    reqSession = requests.Session() # cria sessão para rodar as páginas
    blockSize = 200 # max 200
    d = 1
    s = verificados['lenFnet'][tipoFundo]
    blocoInicial = 0
    fiis0 = {}
    url = getUrl(blockSize, d, s, tipoFundo=codTipoFundo)

    try:
        response = reqSession.get(url, headers=headers, verify=False)
        j = response.json()
    except Exception as e:
        msgTS(f'Erro no download do Json das notícias de {tipoFundo}\t{e}\n{url}')
        continue

    newLenFnet = j['recordsTotal']
    resAtual = newLenFnet - s
    msgTS(f"Total de resultados: {newLenFnet}, {resAtual} em análise\tS: {s}")
    data = j['data']
    if resAtual > blockSize:
        blocos = [_ for _ in range(0, int(resAtual/blockSize) + 1)]
    else:
        blocos = [0]
    # print(blocos)
    if len(blocos) > 0:
        for bloco in tqdm(blocos[1:]):
            [time.sleep(0.1) if bloco > 1 else None]
            d += 1
            s = s + blockSize
            url = getUrl(blockSize, d, s)
            # print(url)
            response = reqSession.get(url, headers=headers, cookies=reqSession.cookies, verify=False)
            j = response.json()
            data = [*data, *j['data']]
    reqSession.close()
    fiAcompanhamento = list(set([*list(cotasAcompanhamento['bia'].keys()),  *list(cotasAcompanhamento['paulo'].keys())]))
    erroTicket = 0
    erro = False

    #################### ANALISA DADOS ######################
    for item in data:
        if item['nomePregao'].strip() in nomesPregao:
            ticketTimer = time.perf_counter()
            ticket = tickets[item['nomePregao'].strip()]
            check = item['id']
            tipoFi = dfFiB3.loc[dfFiB3['ticket'] == ticket, 'subtipo'].reset_index().iloc[0,1]  # FII, Fiagro-Imob, Fiagro-FIDC
            classeFi = dfFiB3.loc[dfFiB3['ticket'] == ticket, 'tipo'].reset_index().iloc[0,1]  # FII ou FIAGRO
            msg = ''
            novoForm = True
            url = f'https://fnet.bmfbovespa.com.br/fnet/publico/exibirDocumento?id={item["id"]}&cvm=true'
            urlDw = url.replace('exibir','download')
            informeMensal, rg, rendimentos, demaisInfos = False, False, False, False
            b3ok = True
            if item["situacaoDocumento"] != "A" or check in fullCheck:  # (repescagem and check in verificados["idTwitter"]) or
                continue
            elif codTipoFundo == 2:
                # FIDC
                sendOk = True
                demaisInfos, msg = getDemaiInfos()
            else:
                # FII
                ticketRend = ''
                sendOk = False
                if item['tipoDocumento'].strip().startswith('Informe Mensal') and item['dataReferencia'] == dataRef:
                    try:
                        informeMensal, msg, dfDadosFundo, dfValorPatrimonialCota, dfAlavancagem = getInformeMensal(msg, dfDadosFundo, dfValorPatrimonialCota, dfAlavancagem)
                        informesToPlan = True  # informeMensal #if not informesToPlan else informesToPlan
                    except Exception as e:
                        erroTicket += 1
                        b3ok = False
                        msgTS(f'Erro RG do {ticket} {e}\n{url}')
                elif item['tipoDocumento'].strip() == 'Relatório Gerencial':
                    sendOk = True
                    rg, msg, dfDadosFundo, ticketsRG = getRG(msg, dfDadosFundo, ticketsRG)
                    rgToPlan = True  # rg if not rgToPlan else rgToPlan
                elif item['tipoDocumento'].strip() == 'Rendimentos e Amortizações':
                    if verificados['ultimoDU'] != str(dt.date.today()): # não envia para o twiiter no último DU
                        sendOk = True
                    try:
                        rendimentos, msg, dfRendimento, rendAmort, ticketRend, novoForm = getRendimentos(msg, dfRendimento, base=base)
                        if rendimentos:
                            try:
                                cotistas = dfDadosFundo.loc[dfDadosFundo['nomePregao'] == ticket, 'cotistas']
                                msg += f"{int(cotistas)} cotistas\n"
                            except Exception as e:
                                msgTS(f'Erro no número de cotistas do {ticket}\n{e}')
                            try:
                                rendimentosToPlan = True if rendAmort['rendimento'][0] else rendimentosToPlan
                            except:
                                rendimentosToPlan = True if rendAmort['rendimento'][ticketRend][0] else rendimentosToPlan
                    except Exception as e:
                        erroTicket += 1
                        b3ok = False
                        msgTS(f'Erro rendimento do {ticket} {e}\n{url}')
                        demaisInfos, msg = getDemaiInfos()
                elif item['tipoDocumento'].strip().startswith('Formulário de Subscrição'):
                    sendOk = True
                    demaisInfos, msg = getInicioSubscricao()
                elif item['tipoDocumento'].strip().startswith('Formulário de Liberação para Negociação'):
                    try:
                        demaisInfos, msg = getConversaoSubscricao()
                    except:
                        demaisInfos, msg = getDemaiInfos()
                elif item['tipoDocumento'].strip().startswith('Informe Trimestral'):
                    demaisInfos, msg = getDemaiInfos()
                    msg = getInformeTrimestral(msg)
                elif item['categoriaDocumento'].strip().startswith("Fato Relevante"):
                    sendOk = True
                    demaisInfos, msg = getDemaiInfos()
                else:
                    demaisInfos, msg = getDemaiInfos()

            if ticket in fiAcompanhamento and check not in verificados["idPessoal"] and b3ok:
                resPessoal, cotasAcompanhamento = sendPessoal(msg, ticketRend)
                if resPessoal:
                    verificados["idPessoal"].append(check)
                    salvajson(verificados, 'verificadosB3')
                    [salvajson(cotasAcompanhamento, 'cotasAcompanhamento') if rendimentos else None]
                else:
                    msgTS('Erro no Telegram - Pessoal')
                    erroTicket += 1
            if ticket in fiagros and check not in verificados["idCanalFiagro"] and b3ok:
                if sendFiagro(msg):
                    verificados["idCanalFiagro"].append(check)
                    salvajson(verificados, 'verificadosB3')
                else:
                    msgTS('Erro no Telegram - Canal Fiagro')
                    erroTicket += 1

            if ticket in lstFIIs and check not in verificados["idCanalFiagro"] and b3ok:
                if sendFii(msg):
                    verificados["idCanalFiagro"].append(check)
                    salvajson(verificados, 'verificadosB3')
                else:
                    msgTS('Erro no Telegram - Canal Fiagro')
                    erroTicket += 1

            # Envia algumas informações para o Twitter
            if check not in verificados["idTwitter"] and b3ok and (ticket in fundosTw or sendOk):
                # código para a API gratuita do Twitter
                if sendTweet(msg, url=url):
                    verificados["idTwitter"].append(check)
                    salvajson(verificados, 'verificadosB3')
                else:
                    erroTicket += 1
                    msgTS('Erro no Twitter')
                twTimer = time.perf_counter()
            elif ticket not in fundosTw:
                # código após o fim da API gratuita do Twitter
                verificados["idTwitter"].append(check)
                salvajson(verificados, 'verificadosB3')

        # s += 1  # Ajuste no fnet
            if erroTicket < 1:
                fullCheck.append(check)
            else:
                erro = True
            erroTicket = 0
            ticketTimer = str(dt.timedelta(seconds=time.perf_counter()-ticketTimer))
            msgTS(f'{ticket} analisado em {ticketTimer}')

    # escreve na planilha
    erroPlan = False
    if codTipoFundo == 1: # FIIs e Fiagros Imobiliários
        if informesToPlan:
            dfDadosFundo = dfDadosFundo[dfDadosFundo['cnpj'] != 0]
            dfValorPatrimonialCota = dfValorPatrimonialCota.astype(float)
            dfAlavancagem  = dfAlavancagem.astype(float)

            # salva DFs
            dfDadosFundo.fillna('-').sort_index().to_hdf('./apoio/dfDadosFundos.h5', key='df')
            dfValorPatrimonialCota.fillna(0).sort_index().to_hdf('./apoio/dfVPC.h5', key='vpc')
            dfAlavancagem.fillna(0).sort_index().to_hdf('./apoio/dfAlavancagem.h5', key='alavancagem')

            dfValorPatrimonialCota = fazUltima(dfValorPatrimonialCota)
            dfAlavancagem = fazUltima(dfAlavancagem)

            dfs.update({'dados': dfDadosFundo.sort_index().reset_index().fillna('-'), # dfDadosFundo.iloc[:, :-2].sort_index().reset_index().fillna('-')
                        'vpc': dfValorPatrimonialCota[dfValorPatrimonialCota.columns[::-1]].sort_index().reset_index().fillna(0),
                        'alavancagem': dfAlavancagem[dfAlavancagem.columns[::-1]].sort_index().reset_index().fillna(0)
            })
        elif rgToPlan:
            # dfDadosFundo = dfDadosFundo[dfDadosFundo['cnpj'] != 0]
            # dfDadosFundo.fillna('-', inplace=True)
            dfDadosFundo.fillna('-').sort_index().to_hdf('./apoio/dfDadosFundos.h5', key='df')
            dfs.update({'dados': dfDadosFundo.sort_index().reset_index().fillna('-')})

        if rendimentosToPlan:
            dfRendimento.iloc[:, :-1] = dfRendimento.iloc[:, :-1].fillna(0) # há um warning se fizer com inplace
            cols = dfRendimento.columns
            dfRendimento.sort_index().fillna(0).to_hdf('./apoio/dfRendimento.h5', key='renda')  # .fillna(0) retirar? era para 2021-10 ser NaN
            dfRendimento['ultima'] = 0
            dfRendimento.loc[dfRendimento.iloc[:, -2] > 0, 'ultima'] = dfRendimento.loc[dfRendimento.iloc[:, -2] > 0, cols[-1]] # pega valores váidos do últomo mês
            dfRendimento.loc[dfRendimento['ultima'] == 0, 'ultima'] = dfRendimento.loc[dfRendimento['ultima'] == 0, cols[-2]] # pega valores váidos da penúltimo mês
            dfRendimento.loc[:, cols[-5:]]= dfRendimento.loc[:, cols[-5:]].replace(0, np.nan)
            dfRendimento['media'] = dfRendimento.loc[:, cols[-4:]].mean(axis=1, skipna=True) # pega valores váidos da última
            # pega valores váidos da penúltima
            dfRendimento.loc[dfRendimento.iloc[:, -3].isnull(), 'media'] = dfRendimento.loc[dfRendimento.iloc[:, -3].isnull(), cols[-5:-1]].mean(axis=1, skipna=True)

            dfs.update({'renda': dfRendimento[dfRendimento.columns[::-1]].fillna(0).sort_index().reset_index()})

        # dfDadosFundo = executaListados(item, dfDadosFundo)
        if dfs:
            try:
                escritorDic(dfs)
                rgToPlan = False
            except Exception as e:
                msgTS(f'Erro no Google Sheets\t{e}')
                erroPlan = True

    # Verifica INFRA
    tempoInfra = tempoInfra + dt.timedelta(minutes=1)
    if (tempoInfra > dt.datetime.now() or tempoInfra.time() > dt.time(22, 6)) and codTipoFundo == 2:
        try:
            verificados = verificaInfra(verificados)
        except Exception as e:
            msgTS(f'Erro na verificação de INFRA\t{e}')

    # salva se a quantidade de notícias analisadas
    if not (erroPlan or erro):
        verificados["lenFnet"][tipoFundo] = newLenFnet  # s
        verificados['fullCheck'] = fullCheck
        if manyTweets:
            verificados['manyTweets'] = manyTweets
        salvajson(verificados, 'verificadosB3')

if hoje > diaAnalise:
    verificados['diaAnalise'] = hoje.strftime('%d/%m/%Y')
    verificados["lenFnet"]["fii"] = 0
    verificados["lenFnet"]["fidc"] = 0
    verificados["idPessoal"] = []
    verificados["idCanalFiagro"] = []
    verificados["idTwitter"] = []
    verificados['fullCheck'] = []
    verificados['manyTweets'] = False
    salvajson(verificados, 'verificadosB3')
tTime = str(dt.timedelta(seconds=time.perf_counter()-tTime))
msgTS(f'Finalizado em {tTime}')

ModuleNotFoundError: ignored